# 1.0 Load Packages & Testing Data

In [1]:
Threads.nthreads()

6

In [2]:
# Load Packages
using Clustering
using ParallelKMeans
using BenchmarkTools
using DelimitedFiles
using mlpack

## Read Data as transposed matrices since Julia is column major

In [3]:
X_1m = permutedims(DelimitedFiles.readdlm("data_1m.csv", ',', Float64));

In [4]:
X_100k = permutedims(DelimitedFiles.readdlm("data_100k.csv", ',', Float64));

In [5]:
X_10k = permutedims(DelimitedFiles.readdlm("data_10k.csv", ',', Float64));

In [6]:
X_1k = permutedims(DelimitedFiles.readdlm("data_1k.csv", ',', Float64));

## Row-wise Data For mlpack

In [7]:
X_1m_mlp = permutedims(X_1m);

In [8]:
X_100k_mlp = permutedims(X_100k);

In [9]:
X_10k_mlp = permutedims(X_10k);

In [10]:
X_1k_mlp = permutedims(X_1k);

# Elbow Method MLPACK

In [11]:
@btime [mlpack.kmeans(i, $X_1m_mlp; algorithm="hamerly", max_iterations=100_000)[1] for i = 2:10];

  254.481 s (93 allocations: 2.08 GiB)


In [12]:
@btime [mlpack.kmeans(i, $X_100k_mlp; algorithm="hamerly", max_iterations=100_000)[1] for i = 2:10];

  18.517 s (93 allocations: 212.88 MiB)


In [13]:
@btime [mlpack.kmeans(i, $X_10k_mlp; algorithm="hamerly", max_iterations=100_000)[1] for i = 2:10];

  794.956 ms (93 allocations: 21.30 MiB)


In [14]:
@btime [mlpack.kmeans(i, $X_1k_mlp; algorithm="hamerly", max_iterations=100_000)[1] for i = 2:10];

  31.211 ms (93 allocations: 2.14 MiB)


# 2.0 Elbow Method Clustering.jl

In [15]:
@btime [Clustering.kmeans($X_1m, i; tol=1e-6, maxiter=100_000).totalcost for i = 2:10];

  653.178 s (21031 allocations: 39.26 GiB)


In [16]:
@btime [Clustering.kmeans($X_100k, i; tol=1e-6, maxiter=100_000).totalcost for i = 2:10];

  45.468 s (14876 allocations: 2.78 GiB)


In [17]:
@btime [Clustering.kmeans($X_10k, i; tol=1e-6, maxiter=100_000).totalcost for i = 2:10]; 

  824.115 ms (4294 allocations: 83.90 MiB)


In [18]:
@btime [Clustering.kmeans($X_1k, i; tol=1e-6, maxiter=100_000).totalcost for i = 2:10]; 

  17.301 ms (767 allocations: 2.24 MiB)


# 3.0 Elbow Method Speed ParallelKMeans.jl

## Lloyd

In [19]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_1m, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  19.955 s (40915 allocations: 140.36 MiB)


In [20]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_100k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  2.758 s (52724 allocations: 17.65 MiB)


In [21]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_10k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  166.957 ms (31942 allocations: 3.80 MiB)


In [22]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_1k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  9.206 ms (9792 allocations: 987.77 KiB)


## Hamerly

In [23]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_1m, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  11.234 s (50446 allocations: 278.97 MiB)


In [24]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_100k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  1.654 s (85374 allocations: 34.73 MiB)


In [25]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_10k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  109.074 ms (51352 allocations: 7.16 MiB)


In [26]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_1k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  12.819 ms (16050 allocations: 1.72 MiB)


# Elkan

In [27]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_1m, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  19.394 s (65605 allocations: 632.12 MiB)


In [28]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_100k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  1.436 s (57010 allocations: 67.54 MiB)


In [29]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_10k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  109.262 ms (48980 allocations: 10.48 MiB)


In [30]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_1k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  13.726 ms (16348 allocations: 2.10 MiB)


## YingYang

In [31]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_1m, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  14.080 s (31669 allocations: 277.54 MiB)


In [32]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_100k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  972.914 ms (24387 allocations: 29.71 MiB)


In [33]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_10k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  95.325 ms (23881 allocations: 4.96 MiB)


In [34]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_1k, i; tol=1e-6, max_iters=100_000, verbose=false).totalcost for i = 2:10];

  9.802 ms (8812 allocations: 1.15 MiB)
